In [1]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [ ]:
import cv2
import mediapipe as mp
import subprocess  # To run osascript commands

# Initialize Mediapipe Hands module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Start capturing video from the first camera device
cap = cv2.VideoCapture(1)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Could not open webcam")
    exit()

pause_video = False  # Initialize the pause_video flag
mute_volume = False  # Initialize the mute_volume flag

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    frame = cv2.flip(frame, 1)

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(rgb_frame)
    

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_finger_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            hand_vertical = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y < hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
            hand_horizontal = abs(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x - hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x) < 0.05

            if hand_vertical and hand_horizontal:
                mute_volume = True
            else:
                mute_volume = False
            


            is_thumb_up = thumb_tip.y < thumb_mcp.y
            is_thumb_down = thumb_tip.y > index_finger_mcp.y
            
            
            # if mute_volume:
            #     print("Back of hand shown! Muting Volume")
            #     subprocess.run(["osascript", "-e", "set volume output volume 0"])
    #        else:
            if is_thumb_up:
                    print("Thumbs Up! Increasing Volume")
                    subprocess.run(["osascript", "-e", "set volume output volume (output volume of (get volume settings) + 10) --100%"])
            elif is_thumb_down:
                    print("Thumbs Down! Decreasing Volume")
                    subprocess.run(["osascript", "-e", "set volume output volume (output volume of (get volume settings) - 10) --100%"])


    # cv2.imshow('Hand Tracking', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
hands.close()
cap.release()
cv2.destroyAllWindows()